<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-11 15:15:49
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.32 C
-------------------
Today PnL: 15.03 K (0.11%)
Current PnL: -22.45 L (-15.27%)
CY Booked + Current PnL: -14.18 L (-9.65%)
-------------------
Total profit:  2.19 L
Total loss:  -24.65 L
-------------------
Total Booked + Current PnL: 12.80 L (10.66%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.24%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.93 L (61.87%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-11.34,M-LC,4.02,218466.0,16986.0,10224.0,2.50,8.43,4.68,13.51,61.0,1.66,1.68,22.09,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,12.64,H-MC,13.89,160764.0,34564.0,11607.0,-2.62,27.39,7.22,36.59,111.0,2.98,1.24,43.33,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-22.38,H-LC,5.72,107648.0,3336.0,13316.0,0.01,3.20,12.37,15.96,11.0,0.25,0.83,20.64,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,93.60,M-SC,10.95,83388.0,-17389.0,17478.0,0.43,-17.25,20.96,0.09,243.0,-0.99,0.64,9.55,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-26.82,H-SC,8.65,218729.0,25709.0,22332.0,0.82,13.32,10.21,24.89,134.0,1.15,1.68,39.77,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.99,H-SC,0.96,101590.0,-910.0,41743.0,-0.84,-0.89,41.09,39.83,116.0,-0.02,0.78,0.00,AR,ATH,MISC
25,DABUR,505.20,735.00,-8.70,H-MC,4.28,198705.0,1677.0,87947.0,0.44,0.85,44.26,45.49,103.0,0.02,1.53,14.00,XY24,BTT,FMCG
58,PGHH,13388.00,18062.58,-30.81,H-MC,6.52,200115.0,-705.0,70821.0,0.89,-0.35,35.39,34.92,82.0,-0.01,1.54,4.31,X40,ATH,FMCG
92,WIPRO,243.46,420.00,-14.70,M-LC,6.66,149798.0,-1147.0,110596.0,1.07,-0.76,73.83,72.51,54.0,-0.01,1.15,5.22,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.87,H-LC,1.99,205925.0,1425.0,39991.0,1.16,0.70,19.42,20.25,10.0,0.04,1.59,9.49,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.06,H-LC,2.42,169551.0,-31835.0,119449.0,0.72,-15.81,70.45,43.51,27.0,-0.27,1.31,18.02,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-8.09,H-LC,3.57,182250.0,-18060.0,78039.0,1.01,-9.02,42.82,29.94,17.0,-0.23,1.40,25.91,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-15.86,H-LC,3.96,129390.0,-136036.0,192946.0,0.86,-51.25,149.12,21.44,8.0,-0.71,1.00,4.64,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.0,-20.06,H-LC,5.51,159687.0,3510.0,22580.0,2.46,2.25,14.14,16.71,19.0,0.16,1.23,19.92,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,11.32,H-SC,13.12,132413.0,7116.0,51893.0,3.79,5.68,39.19,47.09,141.0,0.14,1.02,65.73,MH,ATH,POWER
79,TATAMOTORS,761.77,1065.00,-24.80,M-LC,5.51,237039.0,-39484.0,149548.0,3.05,-14.28,63.09,39.81,55.0,-0.26,1.82,13.60,XY24,NTT,AUTO
85,UNIONBANK,123.87,163.00,-14.73,M-LC,15.14,153279.0,12439.0,32051.0,2.92,8.83,20.91,31.59,67.0,0.39,1.18,37.66,XY24,NTT,BANKS
20,CERA,7989.07,9475.00,-19.33,H-SC,3.07,114930.0,-28873.0,55615.0,2.89,-20.08,48.39,18.60,148.0,-0.52,0.88,26.83,OX40N,NTT,CERAMICS
71,SIEMENS,4406.90,7969.85,-3.04,H-LC,36.10,77902.0,-32270.0,121340.0,2.78,-29.29,155.76,80.85,23.0,-0.27,0.60,14.74,AR,ATH,ELECTRICAL


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.00,104.24,M-SC,3.72,133833.0,-40726.0,40752.0,-6.34,-23.33,30.45,0.02,221.0,-1.00,1.03,16.05,OX40N,NTT,PAINTS
90,VOLTAS,1278.28,1918.49,-10.87,H-MC,8.09,187035.0,-4707.0,100737.0,-4.44,-2.45,53.86,50.08,101.0,-0.05,1.44,4.18,XY25,ATH,AC
66,SAMMAANCAP,170.35,326.00,-160.23,M-SC,4.47,68880.0,-33330.0,126719.0,-3.38,-32.61,183.97,91.37,206.0,-0.26,0.53,11.81,XY25,NTT,FINANCE
81,TITAGARH,1097.23,1548.00,-17.00,H-SC,6.86,136106.0,-55909.0,134799.0,-3.14,-29.12,99.04,41.08,156.0,-0.41,1.05,15.20,XY24,NTT,ENGINEERING
29,EASEMYTRIP,18.12,26.40,-10.53,M-SC,22.94,86163.0,-90652.0,171447.0,-3.07,-51.27,198.98,45.70,197.0,-0.53,0.66,0.00,XY24,NTT,TRAVEL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.60,M-SC,10.95,83388.0,-17389.0,17478.0,0.43,-17.25,20.96,0.09,243.0,-0.99,0.64,9.55,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,83.20,H-SC,20.59,115425.0,-27054.0,46574.0,-3.06,-18.99,40.35,13.70,163.0,-0.58,0.89,36.23,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.33,H-SC,3.07,114930.0,-28873.0,55615.0,2.89,-20.08,48.39,18.60,148.0,-0.52,0.88,26.83,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-69.25,H-SC,2.48,211545.0,-58122.0,94455.0,0.93,-21.55,44.65,13.47,142.0,-0.62,1.63,7.00,XY24,NTT,PAINTS
72,SIS,477.00,528.0,2244.56,H-SC,7.06,63078.0,-18012.0,26682.0,-0.46,-22.21,42.30,10.69,155.0,-0.68,0.49,27.84,OX40N,NTT,MISC


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.52,237120.0,-58814.0,152729.0,0.12,-19.87,64.41,31.73,2.0,-0.39,1.83,1.23,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.75,H-LC,9.82,255630.0,-5971.0,151589.0,0.35,-2.28,59.30,55.67,4.0,-0.04,1.97,3.68,X40,BTT,IT
89,VBL,492.64,671.64,-9.80,H-LC,10.92,274003.0,8963.0,87352.0,2.69,3.38,31.88,36.33,5.0,0.10,2.11,17.13,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.10,H-LC,9.72,196755.0,-19764.0,112662.0,0.46,-9.13,57.26,42.91,7.0,-0.18,1.51,3.04,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-15.86,H-LC,3.96,129390.0,-136036.0,192946.0,0.86,-51.25,149.12,21.44,8.0,-0.71,1.00,4.64,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.64,H-MC,13.89,160764.0,34564.0,11607.0,-2.62,27.39,7.22,36.59,111.0,2.98,1.24,43.33,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.64,H-MC,13.89,160764.0,34564.0,11607.0,-2.62,27.39,7.22,36.59,111.0,2.98,1.24,43.33,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,11.32,H-SC,13.12,132413.0,7116.0,51893.0,3.79,5.68,39.19,47.09,141.0,0.14,1.02,65.73,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,1.98,H-MC,7.66,176060.0,11390.0,56586.0,-2.21,6.92,32.14,41.28,91.0,0.20,1.36,15.45,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-6.49,H-SC,2.62,106305.0,4301.0,39620.0,1.38,4.22,37.27,43.06,123.0,0.11,0.82,22.05,X40N,ATH,MISC
25,DABUR,505.20,735.00,-8.70,H-MC,4.28,198705.0,1677.0,87947.0,0.44,0.85,44.26,45.49,103.0,0.02,1.53,14.00,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.54,159691.0,8778.0,48530.0,-0.81,5.82,30.39,37.97,92.0,0.18,1.23,32.59,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,11.32,H-SC,13.12,132413.0,7116.0,51893.0,3.79,5.68,39.19,47.09,141.0,0.14,1.02,65.73,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-51.84,H-SC,12.85,134562.0,11226.0,122532.0,-0.87,9.10,91.06,108.45,119.0,0.09,1.04,33.49,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-43.95,M-SC,4.46,94732.0,3234.0,75520.0,-0.12,3.53,79.72,86.07,219.0,0.04,0.73,37.04,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-14.70,M-LC,6.66,149798.0,-1147.0,110596.0,1.07,-0.76,73.83,72.51,54.0,-0.01,1.15,5.22,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.36,70176.0,-43373.0,83425.0,1.39,-38.20,118.88,35.27,268.0,-0.52,0.54,71.76,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.67,H-SC,17.43,89668.0,-11295.0,105826.0,1.03,-11.19,118.02,93.63,149.0,-0.11,0.69,33.56,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.51,H-SC,15.88,90435.0,-7545.0,29419.0,-0.31,-7.70,32.53,22.32,152.0,-0.26,0.70,31.16,XR,ATH,FINANCE
84,UJJIVANSFB,52.77,60.00,83.20,H-SC,20.59,115425.0,-27054.0,46574.0,-3.06,-18.99,40.35,13.70,163.0,-0.58,0.89,36.23,OX40N,NTT,BANKS
50,KPIGREEN,497.21,731.35,11.32,H-SC,13.12,132413.0,7116.0,51893.0,3.79,5.68,39.19,47.09,141.0,0.14,1.02,65.73,MH,ATH,POWER


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,492.64,671.64,-9.80,H-LC,10.92,274003.0,8963.0,87352.0,2.69,3.38,31.88,36.33,5.0,0.10,2.11,17.13,X40N,ATH,FMCG
70,SHALBY,261.39,327.00,933.49,M-SC,7.99,140651.0,-40492.0,85966.0,-0.15,-22.35,61.12,25.10,232.0,-0.47,1.08,12.72,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-26.82,H-SC,8.65,218729.0,25709.0,22332.0,0.82,13.32,10.21,24.89,134.0,1.15,1.68,39.77,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.54,159691.0,8778.0,48530.0,-0.81,5.82,30.39,37.97,92.0,0.18,1.23,32.59,AR,ATH,AUTO
55,MEDANTA,1087.93,1486.00,12.64,H-MC,13.89,160764.0,34564.0,11607.0,-2.62,27.39,7.22,36.59,111.0,2.98,1.24,43.33,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.50
1,25,41.18
2,50,70.88


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.44
LC    34.13
MC    26.41
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.99
X40N     15.36
X40      13.43
XY25     10.72
XR        8.97
AR        8.42
OX40N     7.55
X5K       2.23
MH        1.78
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.82
H-SC    24.58
H-MC    23.46
M-SC    13.41
M-LC     7.23
M-MC     2.61
L-SC     1.45
L-LC     1.08
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.37,-10.48,45.99
IT,11.99,-19.84,79.69
FINANCE,8.33,-20.41,64.95
BANKS,7.66,-14.56,63.21
MISC,5.94,-19.07,87.32
PAINTS,5.67,-18.59,41.96
HEALTHCARE,5.23,3.43,20.96
ELECTRICAL,4.71,-14.69,64.36
AUTO,4.42,-17.52,68.65


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2867365.0,23
XR,1127003.0,13
AR,1116113.0,10
X40,828324.0,10
XY25,656315.0,8
X40N,612812.0,12
OX40N,476842.0,10
SR,196418.0,2
X5K,149343.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2824606.0,28
M-SC,1500745.0,17
H-LC,1384645.0,18
H-MC,1329305.0,18
M-LC,438446.0,5
M-MC,338041.0,2
L-SC,272506.0,3
L-MC,57972.0,1
L-LC,46263.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      946109.0      6
M-SC       XY24      875437.0      7
H-SC       AR        672299.0      5
           XR        532226.0      6
H-LC       X40       521859.0      5
H-MC       XY24      484778.0      5
M-MC       XY24      338041.0      2
H-LC       XY25      250966.0      3
           X40N      245062.0      5
           AR        234002.0      2
H-MC       X40       230930.0      4
H-SC       X40N      223840.0      4
H-MC       XY25      222143.0      2
H-SC       OX40N     201821.0      4
           SR        196418.0      2
L-SC       XR        182078.0      2
M-LC       XY24      181599.0      2
M-SC       AR        161282.0      2
H-MC       X40N      143910.0      3
M-LC       X5K       136027.0      1
M-SC       OX40N     135814.0      4
           XY25      126719.0      1
           XR        125958.0      2
H-MC       XR        118173.0      1
M-LC       XR        110596.0      1
L-SC       OX40N      90428.0      1
H-LC       X200       78039.0      1
M-SC       X40        75535.0      1
L-MC       XR         57972.0      1
H-SC       MH         51893.0      1
H-MC       OX40N      48779.0      1
           AR         48530.0      1
L-LC       XY25       46263.0      1
H-LC       XY24       41401.0      1
H-MC       MH         32062.0      1
H-LC       X5K        13316.0      1
M-LC       XY25       10224.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
